In [ ]:
from collections import defaultdict
from bs4 import BeautifulSoup
import os
import json
import trace
from typing import final
import uuid
import requests
import pandas as pd
import streamlit as st
import azure.cognitiveservices.speech as speechsdk
import zipfile
import base64
import threading
import urllib3
import ssl
import openai

ModuleNotFoundError: No module named 'streamlit'

In [ ]:
ssl._create_default_https_context = ssl._create_stdlib_context
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

Load Azure Speech API credentials from environment variables

In [ ]:
subscription_key = "" #insert key
region = "eastus2"
api_url = "https://api.openai.com/v1/chat/completions"
tts_url = "https://learn.microsoft.com/en-us/azure/ai-services/speech-service/language-support?tabs=tts"
AUDIO_FOLDER = '.\\Audio'
ZIP_FOLDER = '.\\Audio'

In [ ]:
def generate_trace_id():
  return str(uuid.uuid4())

TTS

In [ ]:
def fetch_voice_options(url):
  table = pd.read_html(url)
  voice_dict = defaultdict(list)

  for table in tables:
    table.columns = [str(col).strip() for col in table.columns]

    if "Language" in table.columns and "Text to speech voices" in table.columns:
      language_col = "Language"
      voice_col = "Text to speech voices"

      df = table[[language_col, voice_col]].dropna()

      for _, row in df.iterrows():
        lang = row[language_col].strip()
        soup = BeautifulSoup(str(row["Text to speech voices"]), "html.parser")

        list_items = soup.select("l1")
        vocies = [l1.get_text(strip=True) for l1 in list_items]

        if not voices:
          raw_text = soup.get_text(seperator="\n")
          voices = [v.strip() for v in raw_text.split(")") if v.strip()]

        for voice in voices:
          voice = voice.replace("(Female","(Female)").replace("(Male", "(Male)")

      return voice_dict

  st.error("Failed to fetch voice options from the URL.")
  return {}

In [ ]:
def display_tts():
  for key in ['zip_file_ready', 'zip_file_path', 'transcription_file_ready', 'transcription_file_path', 'enable_transcription', 'selected_voices', 'trace_id']:
    if key not in session_state:
      st.session_state[key] = None
  st.markdown("""
  <h1 style='font-size: 2.5rem; margin-bottom: 0.5 rem;>
    Voxcribe
  </h1>""", unsafe_allow_html=True)

  st.markdown("<div style='margin-top: 20px;'></div>", unsafe_allow_html=True)

  st.markdown("""<h3 style='color:#2ecc71;'>Render Multiple Phrases</h3>""", unsafe_allow_html=True)

  uploaded_file = st.file_uploader("Upload a CSV file", key="file_uploader", type=["csv"])

  file_url = 'Audio_Render_Template.csv'
  with open(file_url, "rb") as file:
      csv_file_data = file.read()
      encoded_csv = base64.b64encode(csv_file_data).decode("utf-8")
      data_url = f"data:text/csv;base64,{encoded_csv}"

  if uploaded_file is not None:
    st.success("CSV uploaded successfully! Click ‘Generate Audio & Transcription’ to process.")
    st.button("Generate Audio & Transcription", key="generate_audio_button", on_click=on_generate_audio_click, args=(uploaded_file,))

  st.markdown("""
  <div style="display: flex; align-items: center; text-align: center; margin:15px 0;">
    <hr style="flex:1; border: none; border-top: 1px solid grey;" />
    <span style="padding: 10px; color: lightgrey; font-weight: bold;">OR</span>
    <hr style="flex:1; border: none; border-top: 1px solid grey;" />
  </div>""", unsafe_allow_html=True)

  st.markdown("""<h3 style='color:#2ecc71;'>Render Single Phrase</h3>""", unsafe_allow_html=True)

  input_text = st.text_area("Enter text here:", height=108)
  voice_dict = fetch_voice_options(tts_url)

  selected_language = st.multiselect("Choose a Language", list(voice_dict.keys()))
  selected_voices = {}

  for lang in selected_language:
    voices = voice_dict[lang]
    selected = st.multiselect(f"Choose Talent for {lang}", options=voices, key=f"voices_for_{lang}")

  enable_transcription = st.checkbox("Generate Transcription")

  if 'single_audio_done' not in st.session_state:
    st.session_state['single_audio_done'] = False

  if 'single_audio_path' not in st.session_state:
    st.session_state['single_audio_path'] = ""

  if 'single_transcription_path' not in st.session_state:
    st.session_state['single_transcription_path'] = ""

  if st.button("Render Single Phrase"):
    if input_text.strip() == "":
        st.warning("Enter text")
    elif not selected_voices:
        st.warning("Please select at least one talent")
    else:
        trace_id = str(uuid.uuid4())[:0]
        output_path = os.path.join("single_outputs", f"render_{trace_id}")
        os.makedirs(output_path, exist_ok=True)

        file_name = f"text_{trace_id}"
        text_to_send = input_text

        transcription_row = {
            "File Name": file_name,
            "Text": text_to_send
            }

        for lang, selected_voice_list in selected_voices.items():
          for voice in selected_voice_list:
            language_code = voice.replace("(Female", "").replace("(Male)", "").strip()
            language_directory = os.path.join(output_path, lang)
            os.makedirs(language_directory, exist_ok=True)

            audio_filename = os.path.join(language_directory, f"{file_name}_{language_code}.wav")

            if language_code.strip().startswith("en"):
              api_response, trace_id = text_to_send
            else:
              api_response, trace_id = call_custom_api(api_url, text_to_send, lang, trace_id)

            transcription_row[lang] = api_response
            st.write(lang, language_code)

            success = generate_audio(api_response, audio_filename, lang, language_code)

            if not success:
              st.error(f"Failed to generate for {file_name}")

        # Create ZIP
        zip_filename = os.path.join("single_outputs", f"{trace_id}_audio.zip")
        with zipfile.ZipFile(zip_filename, "w") as zipf:
          for root, _, files in os.walk(output_path):
            for file in files:
              file_path = os.path.join(root, file)
              arcname = os.path.relpath(file_path, output_path)
              zipf.write(file_path, arcname)

        st.session_state['zip_file_ready'] = True
        st.session_state['zip_file_path'] = zip_filename

        st.session_state['enable_transcription'] = enable_transcription
        if enable_transcription:
          # Save transcription to CSV
          transcription_df = pd.DataFrame([transcription_row])
          transcription_filename = os.path.join(output_path, "transcription.csv")
          transcription_df.to_csv(transcription_filename, index=False, encoding='utf-8-sig')
          st.session_state['transcription_file_ready'] = True
          st.session_state['transcription_file_path'] = transcription_filename
        else:
          st.session_state['transcription_file_ready'] = False
          st.session_state['transcription_file_path'] = None

  # Download Buttons
  if st.session_state.get('zip_file_ready') and st.session_state.get('zip_file_path'):
    with open(st.session_state['zip_file_path'], "rb") as f:
      st.download_button("Download Audio ZIP", f, file_name=os.path.basename(st.session_state['zip_file_path']))

  if st.session_state.get('transcription_file_ready') and st.session_state.get('enable_transcription'):
    with open(st.session_state['transcription_file_path'], "rb") as f:
      st.download_button("Download Transcribed CSV", f, file_name="transcription.csv", mime="text/csv")

  # Sidebar Instructions
  st.sidebar.markdown("Instructions:")
  st.sidebar.markdown(f"""
  - To Render Multiple Phrases:
    * Click [here]({data_url}) to download the template CSV file.
    * You can add/delete the 'Language #' columns in the sheet.
    * Upload CSV file using ‘Browse files’.
    * Click ‘Generate Audio & Transcription’.
    * Once processed, a ZIP file will be available to download.
    * Refer [here](https://learn.microsoft.com/en-us/azure/ai-services/speech-service/language-support?tabs=tts) for language/talent support.
  """, unsafe_allow_html=True)

  st.sidebar.markdown(f"""
  - To Render Single Phrase:
    * Enter the phrase.
    * Choose language(s) and associated talent(s).
    * To get transcription, check ‘Generate Transcription’.
    * Click "Render Single Phrase".
    * A ZIP file will be generated for audio(s), and optionally transcription CSV.
  """, unsafe_allow_html=True)

  # Multi Phrase Rendering
  def on_generate_audio_click(uploaded_file):
    voice_dict = fetch_voice_options(tts_url)
    voice_to_language = {}
    for language, voices in voice_dict.items():
      for voice in voices:
          clean_voice = voice.replace("(Female)","").replace("(Male)","").strip()
          voice_to_language[clean_voice] = language

    if uploaded_file is not None:
      trace_id = generate_trace_id()
      output_folder = os.path.join(AUDIO_FOLDER, trace_id)
      os.makedirs(output_folder, exist_ok=True)
      df = pd.read_csv(uploaded_file)

      # The below code will extract all the distict languages and create a folder
      language_columns = [col for col in df.columns if 'Language' in col]
      # Combine all language columns into a single Series
      if language_columns:
        all_languages = pd.concat([df[col] for col in language_columns])
        # Get distint language names
        distinct_languages = all_languages.unique()
      else:
        distinct_languages = []

      transcription_rows = []

      for index, row in df.iterrows():
        file_name = row['File Name']
        text_to_send = row['Text']
        transcription_row = {
          "File Name": file_name,
          "Text": text_to_send
          }

        for col in df.columns:
          language = str(row[col]).replace("(Female)", "").replace("(Male)", "").strip()
          language_name = voice_to_language.get(language, language)

          if language != 'nan':
              language_code = str(language.split("-")[0])

              language_directory = os.path.join(output_folder, language)
              os.makedirs(language_directory, exist_ok=True)

              audio_filename = os.path.join(language_directory, f"{file_name}.wav")

              if language_code.strip().startswith("en"):
                api_response = text_to_send
              else:
                api_response, trace_id = call_custom_api(api_url, text_to_send, language, trace_id)

              if language_name in transcription_row:
                language_name = f"{language_name} ({language})"

              transcription_row[language_name] = api_response

              success = generate_audio(api_response, audio_filename, language_name, language)

              if not success:
                st.error(f"Failed to generate audio for {file_name}")

        transcription_rows.append(transcription_row)

      transcription_df = pd.DataFrame(transcription_rows)
      transcription_filename = os.path.join(output_folder, "transcription.csv")
      transcription_df.to_csv(transcription_filename, index=False, encoding='utf-8-sig')

      zip_filename = create_zip(trace_id, output_folder)

      if zip_filename:
        st.download_button(
          label="Download ZIP File",
          data=open(zip_filename, "rb"),
          file_name=os.path.basename(zip_filename),
          key="download_zip_button"
        )

        st.success("Audio generated successfully!")

In [ ]:
# Function to generate audio
def generate_audio(api_response, audio_filename, language, language_code):
    data_context = api_response
    try:
        # print(data_context)

        # Configure speech synthesis
        speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=region)
        speech_config.speech_synthesis_language = language
        speech_config.speech_synthesis_voice_name = language_code
        speech_config.set_speech_synthesis_output_format(
            speechsdk.SpeechSynthesisOutputFormat.Riff8Khz8BitMonoMULaw
        )

        # Configure audio output (file)
        audio_config = speechsdk.audio.AudioOutputConfig(filename=audio_filename)

        # Create synthesizer and generate speech
        synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

        # Syntesize Speech
        result = synthesizer.speak_text(data_context)
        audio_data = result.audio_data

        with open(audio_filename, "wb") as audio_file:
          audio_file.write(audio_data)

        return True

    except Exception as e:
        print("Error generating audio:", e)
        return False

In [ ]:
def call_custom_api(api_url, text, language, trace_id):
  headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "cache-control": "no-cache"
    }

  data = {
        "model": "gpt-4",
        "messages": [
            {"role": "system", "content": "You are a professional translator."},
            {"role": "user", "content": f"Translate this to {language} literally, word-for-word. Do not add extra words, adapt meanings, or change the structure beyond direct translation of each term. Preserve the exact formatting and placeholders as they appear in the original text:\n\n{text}"}
        ],
        "temperature": 0.1,
        "maxTokens": 500,
        "choiceCount": 1,
        "frequencyPenalty": 0,
        "presencePenalty": 0,
    }

  response = requests.post(api_url, headers=headers, json=data)
  response.raise_for_status()  # raise an error for bad status codes

  content_value = response.json()["choices"][0]["message"]["content"]
  return content_value, trace_id

In [ ]:
def create_zip(trace_id, output_folder):
  zip_filename = os.path.join(ZIP_FOLDER, f"{trace_id}_audio.zip")

  try:
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
    # Iterate through files in output folder and add them to the zip
      for root, _, files in os.walk(output_folder):
        for file in files:
          file_path = os.path.join(root, file)
          arcname = os.path.relpath(file_path, output_folder)
          zipf.write(file_path, arcname)

    return zip_filename

  except Exception as e:
    st.error(f"Failed to create ZIP file: {e}")
    return None

STT

In [ ]:
language_code_map = {
    "English": "en-US",
    "Spanish": "es-ES",
    "French": "fr-FR",
    "German": "de-DE",
    "Chinese": "zh-CN",
    "Japanese": "ja-JP",
    "Korean": "ko-KR",
    "Hindi": "hi-IN",
    "Arabic": "ar-SA",
    "Russian": "ru-RU, ru-UA",
    "Portuguese": "pt-BR",
    "Italian": "it-IT",
    "Dutch": "nl-NL",
    "Swedish": "sv-SE",
    "Norwegian": "no-NO",
    "Finnish": "fi-FI",
    "Danish": "da-DK",
    "Greek": "el-GR",
    "Turkish": "tr-TR",
    "Indonesian": "id-ID",
    "Thai": "th-TH",
    "Vietnamese:": "vi-VN, vi-LA",
    "Malay": "ms-MY",
    "Filipino": "fil-PH",
    "Tagalog": "tl-PH",
    "Romanian": "ro-RO",
    "Ukrainian": "uk-UA",
    "Czech": "cs-CZ",
    "Hungarian": "hu-HU",
    "Polish": "pl-PL",
    "Romanian": "ro-RO",
    "Slovenian": "sl-SI",
    "Bosnian": "bs-BA",
    "Croatian": "hr-HR",
    "Serbian": "sr-RS",
    "Albanian": "sq-AL"}

In [ ]:
# Transcribe audio
def transcribe_audio(filepath, lang=None):
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=region)

    if lang:
        speech_config.speech_recognition_language = lang

    audio_input = speechsdk.AudioConfig(filename=filepath)
    recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)

    all_results = []

    def handle_final_result(evt):
        if evt.result.text:
            all_results.append(evt.result.text)

    recognizer.recognized.connect(handle_final_result)
    recognizer.start_continuous_recognition()

    done = threading.Event()

    def stop_cb(evt):
        done.set()

    recognizer.session_stopped.connect(stop_cb)
    recognizer.canceled.connect(stop_cb)

    done.wait()
    recognizer.stop_continuous_recognition()

    return " ".join(all_results)

# Translate text
def translate_text(text, targets, trace_id):
    translations = []
    for tgt_lang in targets:
        translated, _ = call_custom_api(api_url, text, tgt_lang, trace_id)
        translations.append(translated)
    return translations

# Detect language
def detect_language(text, trace_id):
    prompt = "Identify the language this text is written in and respond with only one word (e.g., English, Spanish)."
    try:
        response, _ = call_custom_api(api_url, prompt, "English", trace_id)
        return response.strip()
    except Exception as e:
        print(f"Language detection failed: {e}")
        return "unknown"

def write_csv(lang_data, output_dir, enable_translation, translate_targets):
  results = {}

  for lang, rows in lang_data.items():
    headers = ["File Name", lang]
    if enable_translation:
        headers += translate_targets

    df = pd.DataFrame(rows, columns=headers)
    csv_path = os.path.join(output_dir, f"{lang}.csv")
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    results[lang] = csv_path

  return results

def process_audio_files(input_paths, autodetect, enable_translation, translate_targets, trace_id):
    base_dir = "stt_temp"
    os.makedirs(base_dir, exist_ok=True)
    output_dir = os.path.join(base_dir, f"csv_output_{trace_id}")
    os.makedirs(output_dir, exist_ok=True)

    lang_data = {}

    for path in input_paths:
        if os.path.isdir(path):
            language_name = os.path.basename(path)
            lang_code = language_code_map.get(language_name.lower())
            files = [f for f in os.listdir(path) if f.endswith(".wav")]

            for f in files:
                audio_path = os.path.join(path, f)
                text = transcribe_audio(audio_path, lang=None if autodetect else lang_code)
                row = [f, text]
                if enable_translation:
                    row += translate_text(text, translate_targets, trace_id)
                lang_data.setdefault(language_name, []).append(row)

        elif os.path.isfile(path) and path.endswith(".wav"):
            filename = os.path.basename(path)
            text = transcribe_audio(path, lang=None if autodetect else None)
            detected_lang = detect_language(text, trace_id) if autodetect else "Unknown"
            row = [filename, text]

            if enable_translation:
                row += translate_text(text, translate_targets, trace_id)
            lang_data.setdefault(detected_lang, []).append(row)

    results = write_csv(lang_data, output_dir, enable_translation, translate_targets)
    return results

def display_stt():
    st.markdown("""
    <h1 style='font-size: 2.5rem; margin-bottom: 0.55rem;'>
    Voxcribe
    </h1>""", unsafe_allow_html=True)

    st.markdown("<div style='margin-top: 20px;'></div>", unsafe_allow_html=True)

    uploaded_zip = st.file_uploader("Upload a ZIP file containing audio folders or files", type=["zip"])

    auto_detect = st.checkbox("Auto Detect Language")
    enable_translation = st.checkbox("Translate")

    translate_targets = []
    if enable_translation:
        translate_targets = st.multiselect("Translate To:", [
          "English", "Spanish","French","German","Chinese","Japanese","Korean","Hindi","Arabic","Russian","Portuguese","Italian","Dutch",
          "Swedish","Norwegian","Finnish","Danish","Greek","Turkish","Indonesian","Thai","Vietnamese:","Malay","Filipino","Tagalog","Romanian",
          "Ukrainian","Czech","Hungarian","Polish","Romanian","Slovenian","Bosnian","Croatian","Serbian","Albanian"
        ])

    if uploaded_zip and st.button("Transcribe Audio"):
        with st.spinner("Expecto Patronum..."):
            trace_id = str(uuid.uuid4())[:8]
            working_dir = os.path.join("stt_temp", trace_id)
            os.makedirs(working_dir, exist_ok=True)

            # Save and unzip uploaded file
            zip_path = os.path.join(working_dir, "input.zip")
            with open(zip_path, "wb") as f:
                f.write(uploaded_zip.read())

            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(working_dir)

            # Analyze folder structure
            folder_contents = os.listdir(working_dir)
            input_paths = [os.path.join(working_dir, name) for name in folder_contents if name != "input.zip"]
            structured = all(os.path.isdir(p) for p in input_paths)

            if not structured and not auto_detect:
              st.error("Unstructured audio files detected. Please enable Auto Detect Language.")
              return

            # Process
            transcription_results = process_audio_files(input_paths, auto_detect, enable_translation, translate_targets, trace_id)

            # ZIP CSVs
            output_zip_path = os.path.join("stt_temp", f"{trace_id}_output.zip")
            with zipfile.ZipFile(output_zip_path, 'w') as zipf:
                for lang, csv_path in transcription_results.values():
                  arcname = f"{lang}.csv"
                  zipf.write(csv_path, arcname)

            # Show Download
            with open(output_zip_path, "rb") as f:
              st.download_button("Download Transcription", data=f, file_name="transcriptions.zip")

if __name__ == "__main__":
  tab1, tab2 = st.tabs(["Text to Speech (TTS)", "Speech to Text (STT)"])
  with tab1:
    display_tts()
  with tab2:
    display_stt()